In [5]:
import pandas as pd
from Bio import SeqIO
from Bio.ExPASy import ScanProsite

So I dun goofed and forgot to have you you guys download sequences as well for this next step, below is a script you help you merge the two files automatically, you'll use a similar pattern for the next set of steps which invovles extracting protien domains. 

In [10]:
#Read your blast results csv in
df = pd.read_csv("C:\\Users\\cmgro\\Downloads\\gadB_genome_feature (1).csv")
#Show the first 5 rows
print(df.head())
#Temp lists to attatch variables 
ids = []
seqs = []
#Seq.IO.parse is. an easy way to read through every gene/prot in a fasta file 
for record in SeqIO.parse("C:\\Users\\cmgro\\Downloads\\gadB_genome_feature (1).fasta", "fasta"):
    # So the fasta ID line is fig|1240239.3.peg.2328|XFF4834R_chr22840|VBIXanFus78442_2328|
    # but we only need the first two elements ("fig and 1240239.3.peg.2328") to make fig|1240239.3.peg.2328
    # We then use this ID to merge back onto the original dataframe (basically we say if this row has this ID, add the sequence to that row)
    
    #Fancy way of extracting the ID
    ids.append("|".join(record.id.split("|")[0:2]))
    #Fancy way of extracting the sequence 
    seqs.append(str(record.seq))
#making a new dataframe from this info 
seq_df = pd.DataFrame.from_dict({"PATRIC ID": ids, "Sequences":seqs})
print(seq_df.head())
#Mergine back and saving 
df = df.merge(seq_df, on="PATRIC ID")
df.to_csv("/Users/cmgro/Downloads/merged_gadB.csv")

                                              Genome   Genome ID  \
0  Photobacterium leiognathi subsp. mandapamensis...  1001530.30   
1        Streptomyces cattleya NRRL 8057 = DSM 46488  1003195.11   
2        Streptomyces cattleya NRRL 8057 = DSM 46488  1003195.29   
3                            Rhodococcus jostii RHA1   101510.16   
4                   Gordonia alkanivorans NBRC 16433  1027371.30   

         Accession                PATRIC ID RefSeq Locus Tag  \
0     BACE01000007    fig|1001530.3.peg.267              NaN   
1         FQ859184   fig|1003195.11.peg.556       SCAT_p0577   
2         CP003229  fig|1003195.29.peg.6962     SCATT_p11640   
3        NC_008268   fig|101510.16.peg.6065     RHA1_ro06016   
4  NZ_BACI01000060   fig|1027371.3.peg.2278  GOALK_060_00450   

          Alt Locus Tag                                         Feature ID  \
0  VBIPhoLei187005_0267    PATRIC.1001530.3.BACE01000007.CDS.3627.5027.rev   
1  VBIStrCat187362_0556   PATRIC.1003195.11.FQ8591

In [7]:
import time
sigs = []

for i in df["Sequences"].values:
    handle = ScanProsite.scan(seq=i, mirror="https://prosite.expasy.org")
    result = ScanProsite.read(handle)
    qq = []
    for i in result:
        print(i)
        qq.append(i["signature_ac"])
    time.sleep(5)
    qq = set(qq)
    sigs.append(qq)
    
df["Prosite results"] = sigs
df.to_csv("/Users/christine/Downloads/gadB_motifs.csv")

{'sequence_ac': 'USERSEQ1', 'start': 266, 'stop': 287, 'signature_ac': 'PS00392', 'level_tag': '(0)'}
{'sequence_ac': 'USERSEQ1', 'start': 267, 'stop': 288, 'signature_ac': 'PS00392', 'level_tag': '(0)'}


KeyboardInterrupt: 

In [9]:
df


,Genome,Genome ID,Accession,PATRIC ID,RefSeq Locus Tag,Alt Locus Tag,Feature ID,Annotation,Feature Type,Start,...,Strand,FIGfam ID,PATRIC genus-specific families (PLfams),PATRIC cross-genus families (PGfams),Protein ID,AA Length,Gene Symbol,Product,GO,Sequences
0,Photobacterium leiognathi subsp. mandapamensis...,1001530.30,BACE01000007,fig|1001530.3.peg.267,NaN,VBIPhoLei187005_0267,PATRIC.1001530.3.BACE01000007.CDS.3627.5027.rev,PATRIC,CDS,3627,...,-,FIG01376814,PLF_657_00002493,PGF_00008094,NaN,466,NaN,Glutamate decarboxylase (EC 4.1.1.15),NaN,MALHEVKRQNENDPIFGSDIINNVAATTKLPQHEQAPNVIYQMIRD...
1,Streptomyces cattleya NRRL 8057 = DSM 46488,1003195.11,FQ859184,fig|1003195.11.peg.556,SCAT_p0577,VBIStrCat187362_0556,PATRIC.1003195.11.FQ859184.CDS.643114.644556.rev,PATRIC,CDS,643114,...,-,FIG01376814,PLF_1883_00001091,PGF_00008094,CCB71603.1,480,NaN,Glutamate decarboxylase (EC 4.1.1.15),NaN,MPLKHPHHQADHPGHGNRDIEVNPIFAREPLSVPRYALPSGEMEPE...
2,Streptomyces cattleya NRRL 8057 = DSM 46488,1003195.29,CP003229,fig|1003195.29.peg.6962,SCATT_p11640,NaN,PATRIC.1003195.29.CP003229.CDS.1168546.1169988...,PATRIC,CDS,1168546,...,+,FIG01376814,PLF_1883_00001091,PGF_00008094,NaN,480,NaN,Glutamate decarboxylase (EC 4.1.1.15),GO:0004351|glutamate decarboxylase activity,MPLKHPHHQADHPGHGNRDIEVNPIFAREPLSVPRYALPSGEMEPE...
3,Rhodococcus jostii RHA1,101510.16,NC_008268,fig|101510.16.peg.6065,RHA1_ro06016,VBIRhoJos26306_6065,PATRIC.101510.16.NC_008268.CDS.6469054.6470436...,PATRIC,CDS,6469054,...,+,FIG01376814,PLF_1827_00001126,PGF_00008094,YP_705951.1,460,gadA,Glutamate decarboxylase (EC 4.1.1.15),NaN,MSKHHHHHRDVISPAYTGRLSIDPFPALRLPDEETDPEAAYRFIHD...
4,Gordonia alkanivorans NBRC 16433,1027371.30,NZ_BACI01000060,fig|1027371.3.peg.2278,GOALK_060_00450,VBIGorAlk198381_2278,PATRIC.1027371.3.NZ_BACI01000060.CDS.48076.494...,PATRIC,CDS,48076,...,-,FIG01376814,PLF_2053_00000116,PGF_00008094,ZP_08765886.1,469,NaN,Glutamate decarboxylase (EC 4.1.1.15),NaN,MTPKSNSQARATTNIGDNLNLRTVFTRPGEATDLPKFSMADDMLLP...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Streptomyces griseoaurantiacus M045,996637.30,NZ_AEYX01000032,fig|996637.3.peg.2519,SGM_2301,VBIStrGri184913_2519,PATRIC.996637.3.NZ_AEYX01000032.CDS.64950.6638...,PATRIC,CDS,64950,...,+,FIG01376814,PLF_1883_00001091,PGF_00008094,ZP_08286809.1,476,NaN,Glutamate decarboxylase (EC 4.1.1.15),NaN,MPEQHAGDDDRASRDLEINPVFSREPLRVPRYALPDAEMEPDTAYQ...
496,Fusobacterium nucleatum subsp. animalis ATCC 5...,997347.40,AFQD01000083,fig|997347.4.peg.495,HMPREF9094_0533,VBIFusNuc185587_0495,PATRIC.997347.4.AFQD01000083.CDS.2159.3436.rev,PATRIC,CDS,2159,...,-,FIG01376814,PLF_848_00001773,PGF_00008094,EGQ80440.1,425,gadB,Glutamate decarboxylase (EC 4.1.1.15),NaN,MLPETAYQIVHDESMLDGNARLNLATFVSTWMDESANKLYSEAFDK...
497,Halomonas sp. TD01,999141.50,NZ_AFQW01000044,fig|999141.5.peg.1971,GME_10021,VBIHalSp187213_1971,PATRIC.999141.5.NZ_AFQW01000044.CDS.35846.3722...,PATRIC,CDS,35846,...,-,FIG01376814,PLF_2745_00005740,PGF_00008094,ZP_08637023.1,459,NaN,Glutamate decarboxylase (EC 4.1.1.15),NaN,MTNRSHSMDDEIYGSSTLAKPLPKSGFPDDEQSPRWVCAAVRDELM...
498,Salinispora pacifica DSM 45543,999542.30,AQZB01000001,fig|999542.3.peg.2271,NaN,VBISalPac229648_2271,PATRIC.999542.3.AQZB01000001.CDS.2472511.24739...,PATRIC,CDS,2472511,...,+,FIG01376814,PLF_168694_00002307,PGF_00008094,NaN,466,NaN,Glutamate decarboxylase (EC 4.1.1.15),NaN,MPDMEQDFDMIASDSLLLNPMFARPGQSTELTRFTIPDAATLPETA...
